# question 7

# AMAZON ecommerce website

In [1]:
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

# a. Scrap the selected website and collect all the product related information from the website and store them in a data frame.

In [11]:
number_of_pages = 2

def scrap_AMAZON_ecommerce_website(number_of_pages):  
    page = requests.get('https://www.amazon.in/gp/bestsellers/electronics/ref=zg_bs_nav_0'+str(number_of_pages)+'?ie=UTF8&pg='+str(number_of_pages))
    page_content = page.content
    page_soup = BeautifulSoup(page_content)
 
    productDetails_List = []
    
    for detail in page_soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        product_name = detail.find('span', attrs={'class':'zg-text-center-align'})
        name = product_name.find_all('img', alt=True)
        product_rating = detail.find('span', attrs={'class':'a-icon-alt'})
        users_rated = detail.find('a', attrs={'class':'a-size-small a-link-normal'})
        product_price = detail.find('span', attrs={'class':'p13n-sc-price'})

        product_list=[]

        if product_name is not None:
            product_list.append(name[0]['alt'])
        else:
            product_list.append("unknown-product")

            
        if product_rating is not None:
            product_list.append(product_rating.text)
        else:
            product_list.append('0')

            
        if users_rated is not None:
            product_list.append(users_rated.text)
        else:
            product_list.append('0')     

        if product_price is not None:
            product_list.append(product_price.text)
        else:
            product_list.append('0')
            
            
        productDetails_List.append(product_list)    
    return productDetails_List

In [12]:
output = []
for j in range(1, number_of_pages+1):
    output.append(scrap_AMAZON_ecommerce_website(j))
y = lambda x: [item for sublist in x for item in sublist]
product_dataframe = pd.DataFrame(y(output),columns=['Product Name','Ratings','Customers_Reviews', 'Product_Price'])
product_dataframe.to_csv('amazon_products.csv', index=False,encoding='utf-8-sig')
product_dataframe.head()

,Product Name,Ratings,Customers_Reviews,Product_Price
0,"Redmi Note 9 (Aqua Green, 4GB RAM, 64GB Storag...",4.2 out of 5 stars,715,"₹ 11,998.00"
1,"Redmi Note 9 (Pebble Grey, 4GB RAM 64GB Storag...",4.2 out of 5 stars,904,"₹ 11,998.00"
2,"Redmi Note 9 (Arctic White, 4GB RAM, 64GB Stor...",4.3 out of 5 stars,842,"₹ 11,999.00"
3,Fire TV Stick streaming media player with Alex...,4.2 out of 5 stars,"30,201","₹ 2,599.00"
4,boAt BassHeads 100 in-Ear Wired Earphones with...,4.0 out of 5 stars,"85,143",₹ 379.00


In [13]:
# Ratings are given out of 5,  
# in here keep only the rating value and remove other extra parts.

product_dataframe['Ratings'] = product_dataframe['Ratings'].apply(lambda x: x.split()[0])
product_dataframe['Ratings'] = pd.to_numeric(product_dataframe['Ratings'])


In [14]:
# From the product price column, remove all the ₹ symbol, comma( , ) and split it by (.)

product_dataframe["Product_Price"] = product_dataframe["Product_Price"].str.replace('₹', '')
product_dataframe["Product_Price"] = product_dataframe["Product_Price"].str.replace(',', '')
product_dataframe['Product_Price'] = product_dataframe['Product_Price'].apply(lambda x: x.split('.')[0])
product_dataframe['Product_Price'] = product_dataframe['Product_Price'].astype(int)

In [15]:
# From the Customers_Reviews column, remove all the commas.

product_dataframe["Customers_Reviews"] = product_dataframe["Customers_Reviews"].str.replace(',', '')
product_dataframe['Customers_Reviews'] = pd.to_numeric(product_dataframe['Customers_Reviews'], errors='ignore')


In [16]:
product_dataframe

,Product Name,Ratings,Customers_Reviews,Product_Price
0,"Redmi Note 9 (Aqua Green, 4GB RAM, 64GB Storag...",4.2,715,11998
1,"Redmi Note 9 (Pebble Grey, 4GB RAM 64GB Storag...",4.2,904,11998
2,"Redmi Note 9 (Arctic White, 4GB RAM, 64GB Stor...",4.3,842,11999
3,Fire TV Stick streaming media player with Alex...,4.2,30201,2599
4,boAt BassHeads 100 in-Ear Wired Earphones with...,4.0,85143,379
...,...,...,...,...
95,Havells 6A Four-Way Extension Board (White),4.1,6122,379
96,Boult Audio Bass Buds Q2 Over-Ear Wired Lightw...,5.0,4,699
97,"Redmi Note 9 Pro (Aurora Blue, 6GB RAM, 128GB ...",4.3,11975,16999
98,boAt 100 Wireless On-Neck Bluetooth V5.0 Earph...,3.9,631,1299


In [17]:
product_dataframe.dtypes

Product Name          object
Ratings              float64
Customers_Reviews      int64
Product_Price          int32
dtype: object

# i.	Top 10 most popular products based on sales

In [7]:
top_10_products = product_dataframe.sort_values(['Ratings'],axis=0, ascending=False)[:10]
top_10_products


,Product Name,Ratings,Customers_Reviews,Product_Price
96,Boult Audio Bass Buds Q2 Over-Ear Wired Lightw...,5.0,4,699
15,"Redmi 9A (Sea Blue, 2GB RAM, 32GB Storage)",4.5,166,6799
27,"Redmi 9A (Nature Green, 3GB RAM, 32GB Storage)",4.5,3,7499
39,"Redmi Note 9 (Scarlet Red, 4GB RAM, 64GB Stora...",4.4,633,11999
60,"ELV Desktop Cell Phone Stand Tablet Stand, Alu...",4.4,10313,149
90,SanDisk 32GB Class 10 Micro SDHC Memory Card w...,4.4,269802,469
25,"ELV Desktop Cell Phone Stand Tablet Stand, Alu...",4.4,10313,149
8,"Redmi 9A (Nature Green, 2GB RAM, 32GB Storage)",4.4,171,6799
66,SanDisk 64GB Class 10 microSDXC Memory Card wi...,4.4,269802,809
23,SanDisk Cruzer Blade 32GB USB Flash Drive,4.3,92884,349


# ii.	Top 10 most popular products based on customer reviews.

In [8]:
top_10_products = product_dataframe.sort_values(["Customers_Reviews"], axis=0, ascending=False)[:10]
top_10_products

,Product Name,Ratings,Customers_Reviews,Product_Price
90,SanDisk 32GB Class 10 Micro SDHC Memory Card w...,4.4,269802,469
66,SanDisk 64GB Class 10 microSDXC Memory Card wi...,4.4,269802,809
93,boAt Rockerz 255 Sports Wireless Headset with ...,3.9,126669,1199
64,boAt Rockerz 255 Pro Wireless Headset with ASA...,3.9,126669,1499
18,boAt Rockerz 255 Sports Wireless Headset with ...,3.9,126669,1199
30,boAt BassHeads 225 in-Ear Wired Earphones with...,3.9,102444,549
23,SanDisk Cruzer Blade 32GB USB Flash Drive,4.3,92884,349
51,"Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 64GB ...",4.3,86912,16499
7,"Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB...",4.3,86912,17499
91,"Samsung Galaxy M31 (Space Black, 6GB RAM, 64GB...",4.3,86912,16499


# FLIPCART ecommerce website

In [19]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd

flipcart_LINK = 'https://www.flipkart.com/search?q=oppo&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
webpage = uReq(flipcart_LINK)
page_html = webpage.read()
webpage.close()

page_soup = soup(page_html, "html.parser")
page_containers = page_soup.findAll("div",{"class":"_3O0U0u"}) 

product =[]; product_price =[]; product_discount =[]; product_review =[]; product_rate = []; product_discription =[]

for element in page_containers:
    
    title_element = element.findAll("div",{"class":"_3wU53n"})
    product_name = title_element[0].text
    price_element = element.findAll("div",{"class":"_1vC4OE _2rQ-NK"})
    discount_element = element.findAll("div",{"class":"VGWI6T"})
    review_element = element.findAll("span",{"class":"_38sUEc"})
    rate_element = element.findAll('div',{'class':'hGSR34'})
    discription_element = element.findAll("ul",{"class":"vFw0gD"})
    
    try:
        price = price_element[0].text
        discount= discount_element[0].text
        rate = rate_element[0].text
        review = review_element[0].text
        discription = discription_element[0].text
        
    except:
        price = 'No Data'
        discount = 'No Data'
        review = 'No Data'
        rate = 'No Data'
        discription = 'No Data'
        
    product.append(product_name)
    product_price.append(price)
    product_discount.append(discount)
    product_review.append(review)
    product_rate.append(rate)
    product_discription.append(discription) 
    
  
    print("Product : " + product_name)
    print("Price : " + price)
    print("discount :"+discount)
    print("Rate : "+rate)
    print("Review :"+review)
    print("discrption :"+discription)
    print()
    print('-----------------------------------------------------------------------------------------')



Product : OPPO A9 2020 (Space Purple, 128 GB)
Price : ₹13,990
discount :26% off
Rate : 4.3
Review :71,109 Ratings & 6,591 Reviews
discrption :4 GB RAM | 128 GB ROM | Expandable Upto 256 GB16.51 cm (6.5 inch) Display48MP + 8MP + 2MP + 2MP | 16MP Front Camera5000 mAh BatteryQualcomm Snapdragon 665 ProcessorBrand Warranty of 1 Year Available for Mobile and 6 Months for Accessories

-----------------------------------------------------------------------------------------
Product : OPPO A9 2020 (Marine Green, 128 GB)
Price : ₹17,990
discount :18% off
Rate : 4.3
Review :32,083 Ratings & 2,990 Reviews
discrption :8 GB RAM | 128 GB ROM | Expandable Upto 256 GB16.51 cm (6.5 inch) Display48MP + 8MP + 2MP + 2MP | 16MP Front Camera5000 mAh BatteryQualcomm Snapdragon 665 ProcessorBrand Warranty of 1 Year Available for Mobile and 6 Months for Accessories

-----------------------------------------------------------------------------------------
Product : OPPO F17 Pro (Matte Black, 128 GB)
Price : ₹22

In [20]:
flipcart_df = pd.DataFrame({'Product Name':product ,'Price':product_price,'discount':product_discount,'Rate':product_rate,'Review':product_review ,'Discription':product_discription})
flipcart_df.to_csv('Flipcart_Product.csv',encoding='utf-8-sig')
flipcart_df.head()


,Product Name,Price,discount,Rate,Review,Discription
0,"OPPO A9 2020 (Space Purple, 128 GB)","₹13,990",26% off,4.3,"71,109 Ratings & 6,591 Reviews",4 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
1,"OPPO A9 2020 (Marine Green, 128 GB)","₹17,990",18% off,4.3,"32,083 Ratings & 2,990 Reviews",8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
2,"OPPO F17 Pro (Matte Black, 128 GB)","₹22,990",11% off,4.4,522 Ratings & 61 Reviews,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
3,"OPPO F17 Pro (Metallic White, 128 GB)","₹22,990",11% off,4.4,522 Ratings & 61 Reviews,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
4,"OPPO A12 (Black, 32 GB)","₹9,990",9% off,4.3,"15,320 Ratings & 1,202 Reviews",3 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...


In [21]:

flipcart_df["Price"] = flipcart_df["Price"].str.replace('₹', '')
flipcart_df["Price"] = flipcart_df["Price"].str.replace(',', '')
flipcart_df['Price'] = flipcart_df['Price'].apply(lambda x: x.split('.')[0])
flipcart_df['Price'] = flipcart_df['Price'].astype(int)


flipcart_df["Review"] = flipcart_df["Review"].str.replace(',', '')
flipcart_df['Review'] = pd.to_numeric(flipcart_df['Review'], errors='ignore')
flipcart_df['Review'] = flipcart_df['Review'].apply(lambda x: x.split()[0])
flipcart_df['Review'] = pd.to_numeric(flipcart_df['Review'])

flipcart_df.head()

,Product Name,Price,discount,Rate,Review,Discription
0,"OPPO A9 2020 (Space Purple, 128 GB)",13990,26% off,4.3,71109,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
1,"OPPO A9 2020 (Marine Green, 128 GB)",17990,18% off,4.3,32083,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
2,"OPPO F17 Pro (Matte Black, 128 GB)",22990,11% off,4.4,522,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
3,"OPPO F17 Pro (Metallic White, 128 GB)",22990,11% off,4.4,522,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
4,"OPPO A12 (Black, 32 GB)",9990,9% off,4.3,15320,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...


In [22]:
#i.	Top 10 most popular products based on customer reviews

flipcart_top10 = flipcart_df.sort_values(["Rate"], axis=0, ascending=False)[:10]
flipcart_top10

,Product Name,Price,discount,Rate,Review,Discription
2,"OPPO F17 Pro (Matte Black, 128 GB)",22990,11% off,4.4,522,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
3,"OPPO F17 Pro (Metallic White, 128 GB)",22990,11% off,4.4,522,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
5,"OPPO Reno2 F (Lake Green, 128 GB)",21990,24% off,4.4,3621,8 GB RAM | 128 GB ROM16.51 cm (6.5 inch) Displ...
6,"OPPO Reno2 F (Sky White, 256 GB)",17990,10% off,4.4,10681,6 GB RAM | 256 GB ROM16.51 cm (6.5 inch) Displ...
9,"OPPO F17 Pro (Magic Blue, 128 GB)",22990,11% off,4.4,522,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
20,"OPPO F11 Pro (Aurora Green, 128 GB)",14990,50% off,4.4,121981,6 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
15,"OPPO Reno2 F (Lake Green, 256 GB)",17990,10% off,4.4,10681,6 GB RAM | 256 GB ROM16.51 cm (6.5 inch) Displ...
0,"OPPO A9 2020 (Space Purple, 128 GB)",13990,26% off,4.3,71109,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
21,"OPPO A12 (Black, 64 GB)",11490,4% off,4.3,3063,4 GB RAM | 64 GB ROM | Expandable Upto 256 GB1...
18,"OPPO A5s (Red, 32 GB)",8990,30% off,4.3,20433,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...


In [23]:
#ii.	Top 10 most popular products based on customer reviews

flipcart_top10 = flipcart_df.sort_values(["Review"], axis=0, ascending=False)[:10]
flipcart_top10

,Product Name,Price,discount,Rate,Review,Discription
20,"OPPO F11 Pro (Aurora Green, 128 GB)",14990,50% off,4.4,121981,6 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
0,"OPPO A9 2020 (Space Purple, 128 GB)",13990,26% off,4.3,71109,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
10,"OPPO A9 2020 (Marine Green, 128 GB)",13990,26% off,4.3,71109,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
11,"OPPO A9 2020 (Vanilla Mint, 128 GB)",13990,26% off,4.3,71109,4 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
1,"OPPO A9 2020 (Marine Green, 128 GB)",17990,18% off,4.3,32083,8 GB RAM | 128 GB ROM | Expandable Upto 256 GB...
18,"OPPO A5s (Red, 32 GB)",8990,30% off,4.3,20433,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...
17,"OPPO A5s (Black, 32 GB)",8990,30% off,4.3,20433,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...
16,"OPPO A5s (Blue, 32 GB)",8990,30% off,4.3,20433,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...
4,"OPPO A12 (Black, 32 GB)",9990,9% off,4.3,15320,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...
8,"OPPO A12 (Blue, 32 GB)",9990,9% off,4.3,15320,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB1...
